In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro, stats
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson
#pd.options.display.float_format = '{:.2f}'.format


## Čistenie

In [ ]:
df = pd.read_csv("macro-economic-db-quarterly.csv", delimiter=";", index_col=0)
df = df.transpose()

df.reset_index(inplace=True)
df.rename(columns={"index":"stvrtrok"}, inplace=True)
for column in df.columns[1:]:
    df[column] = df[column].str.replace(",", "").astype(float)

columns_to_modify = df.columns.difference(["urok"])
df[columns_to_modify] = df[columns_to_modify].replace(0, np.nan)

df["bilancia"] = df.vyvoz - df.dovoz

df["NaN_Percentage"] = df.isna().sum(axis=1) / df.shape[1] * 100
df_cut =  df[df["NaN_Percentage"] <= 30].drop(columns=["NaN_Percentage"])
df_cut.reset_index(drop=True, inplace=True)

df_clean = df_cut.bfill()
df_clean = df_clean.ffill()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["NaN_Percentage"], marker='o', linestyle='-', label="NaN%")


plt.axhline(y=30, color='r', linestyle='--', label="30%")


plt.xlabel("Riadok")
plt.ylabel("Percento Chýbajúcich Údajov")
plt.title("Percento Chýbajúcich Údajov za Riadok (s 30% Prahom)")
plt.legend()
plt.grid(True)


plt.show()


# Modelovanie

## HDP

In [ ]:
#Rozdelenie a štandardizácia dát
X = df_clean[[ "m2", "nezamestnanost", "urok", "olej", "bilancia", "tvorba_kapitalu"]]
y = df_clean.hdp
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

In [ ]:

# Generate synthetic dataset


# Define models
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=0.1),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.9)
}

# DataFrame to store results
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    # Shapiro-Wilk test for normality
    shapiro_stat, shapiro_p = shapiro(residuals)
    
    # Breusch-Pagan test for heteroscedasticity
    X_with_const = sm.add_constant(X_train)  # Add constant term
    _, bp_pvalue, _, _ = het_breuschpagan(residuals, X_with_const)
    
    # Durbin-Watson test for autocorrelation
    dw_stat = durbin_watson(residuals)
    
    # Store results
    results.append({
        "Model": name,
        "SW p-hodnota": shapiro_p,
        "BP p-hodnota": bp_pvalue,
        "DW štatistika": dw_stat
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

results_df


In [ ]:
#Lineárna regresia
mlr = LinearRegression()
mlr.fit(X_train, y_train)
y_pred_test = mlr.predict(X_test)
y_pred_train = mlr.predict(X_train)

np.set_printoptions(precision=2)
print(np.concatenate((np.array(y_pred_test).reshape(len(y_pred_test), 1), np.array(y_test).reshape(len(y_test), 1)), 1))

r2_test = mlr.score(X_test, y_test)
print(f"R²: {r2_test}")


residuals = y_train - y_pred_train


fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(residuals, bins=20, kde=True, ax=axs[0])
axs[0].axvline(0, color='red', linestyle='dashed') 
axs[0].set_title("Histogram of Reziduí")
axs[0].set_xlabel("Reziduá")
axs[0].set_ylabel("Počet")


axs[1].scatter(y_pred_train, residuals, alpha=0.7, edgecolor='k')
axs[1].axhline(y=0, color='red', linestyle='dashed')
axs[1].set_title("Reziduá vs. Predikované hodnoty")
axs[1].set_xlabel("Predikované hodnoty")
axs[1].set_ylabel("Reziduá")

plt.tight_layout()
plt.show()


stat, p = shapiro(residuals)
print(f"Shapiro-Wilk Test: Stat={stat:.4f}, p={p:.4f}")
if p > 0.05:
    print("Reziduá sú približne normálne.")
else:
    print("Reziduá sa signifikantne odchyľujú od normnality.")


X_test_with_const = sm.add_constant(X_train)
bp_test_stat, bp_test_pval, _, _ = het_breuschpagan(residuals, X_test_with_const)


print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {bp_test_stat:.4f}")
print(f"p-value: {bp_test_pval:.4f}")


if bp_test_pval < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


In [ ]:
#OLS

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)


X_train_const = sm.add_constant(X_train)


ols_model = sm.OLS(y_train, X_train_const).fit()


print(ols_model.summary())


In [ ]:
#Ridge
ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X_train, y_train)


y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)


ridge_r2_train = ridge_model.score(X_train, y_train)
ridge_r2_test = ridge_model.score(X_test, y_test)

print(f"Ridge Regresia R² (Train): {ridge_r2_train:.4f}")
print(f"Ridge Regresia R² (Test): {ridge_r2_test:.4f}")


ridge_residuals = y_train - y_train_pred




X_with_const = sm.add_constant(X_train)
ridge_bp_test = het_breuschpagan(ridge_residuals, X_with_const)
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {ridge_bp_test[0]}, p-value: {ridge_bp_test[1]}")
if ridge_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(ridge_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(ridge_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(ridge_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Lasso
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)


y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)


lasso_r2_train = lasso_model.score(X_train, y_train)
lasso_r2_test = lasso_model.score(X_test, y_test)

print(f"Lasso Regresia R² (Train): {lasso_r2_train:.4f}")
print(f"Lasso Regresia R² (Test): {lasso_r2_test:.4f}")


lasso_residuals = y_train - y_train_pred




lasso_bp_test = het_breuschpagan(lasso_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test (Lasso Regression):")
print(f"Test Statistic: {lasso_bp_test[0]}, p-value: {lasso_bp_test[1]}")
if lasso_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(lasso_residuals)
print("\nShapiro-Wilk Test for Normality of Residuals:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(lasso_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(lasso_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()



In [ ]:
#Elastic net
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.9) 
elastic_model.fit(X_train, y_train)


y_train_pred = elastic_model.predict(X_train)
y_test_pred = elastic_model.predict(X_test)


elastic_r2_train = elastic_model.score(X_train, y_train)
elastic_r2_test = elastic_model.score(X_test, y_test)

print(f"Elastic Net R² (Train): {elastic_r2_train:.4f}")
print(f"Elastic Net R² (Test): {elastic_r2_test:.4f}")


elastic_residuals = y_train - y_train_pred





elastic_bp_test = het_breuschpagan(elastic_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {elastic_bp_test[0]}, p-value: {elastic_bp_test[1]}")
if elastic_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(elastic_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")


fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(elastic_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(elastic_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()

In [ ]:
#Residuá
models = {
    "Lineárna Regresia": mlr,
    "Ridge": ridge_model,
    "Lasso": lasso_model,
    "ElasticNet": elastic_model
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, (name, model) in zip(axes, models.items()):
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    ax.scatter(y_pred, residuals, alpha=0.7, edgecolor='k')
    ax.axhline(y=0, color='red', linestyle='dashed') 
    ax.set_title(name)
    ax.set_xlabel("Predikované hodnoty")
    ax.set_ylabel("Reziduá")


plt.tight_layout()
plt.show()


In [ ]:
# Q-Q graf
residuals_lr = y_train - mlr.predict(X_train)
residuals_ridge = y_train - ridge_model.predict(X_train)
residuals_lasso = y_train - lasso_model.predict(X_train)
residuals_elastic = y_train - elastic_model.predict(X_train)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle("Q-Q Grafy", fontsize=16)

stats.probplot(residuals_lr, dist="norm", plot=axes[0, 0])
axes[0, 0].set_title("Lineárna Regresia")


stats.probplot(residuals_ridge, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title("Ridge")


stats.probplot(residuals_lasso, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title("Lasso")


stats.probplot(residuals_elastic, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title("ElasticNet")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
#Hodnotiace metriky
y_pred_train_lr = mlr.predict(X_train)  
y_pred_test_lr = mlr.predict(X_test)    

y_pred_train_ridge = ridge_model.predict(X_train)  
y_pred_test_ridge = ridge_model.predict(X_test)    

y_pred_train_lasso = lasso_model.predict(X_train)  
y_pred_test_lasso = lasso_model.predict(X_test)    

y_pred_train_elastic = elastic_model.predict(X_train)  
y_pred_test_elastic = elastic_model.predict(X_test)    


mse_test_lr = mean_squared_error(y_test, y_pred_test_lr)
mae_test_lr = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_lr = np.sqrt(mse_test_lr)
mape_test_lr = mean_absolute_percentage_error(y_test, y_pred_test_lr)

mse_test_ridge = mean_squared_error(y_test, y_pred_test_ridge)
mae_test_ridge = mean_absolute_error(y_test, y_pred_test_ridge)
rmse_test_ridge = np.sqrt(mse_test_ridge)
mape_test_ridge = mean_absolute_percentage_error(y_test, y_pred_test_ridge)

mse_test_lasso = mean_squared_error(y_test, y_pred_test_lasso)
mae_test_lasso = mean_absolute_error(y_test, y_pred_test_lasso)
rmse_test_lasso = np.sqrt(mse_test_lasso)
mape_test_lasso = mean_absolute_percentage_error(y_test, y_pred_test_lasso)

mse_test_elastic = mean_squared_error(y_test, y_pred_test_lr)
mae_test_elastic = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_elastic = np.sqrt(mse_test_elastic)
mape_test_elastic = mean_absolute_percentage_error(y_test, y_pred_test_elastic)



r2_lr = r2_test
r2_ridge = ridge_r2_test
r2_lasso = lasso_r2_test
r2_elastic = elastic_r2_test

metrics = {
    "Model": [
        "Lineárna Regresia",
        "Ridge",
        "Lasso",
        "ElasticNet"
    ],
    "MSE": [
        mse_test_lr,
        mse_test_ridge,
        mse_test_lasso,
        mse_test_elastic
    ],
    "MAE": [
        mae_test_lr,
        mae_test_ridge,
        mae_test_lasso,
        mae_test_elastic
    ],
    "RMSE": [
        rmse_test_lr,
        rmse_test_ridge,
        rmse_test_lasso,
        rmse_test_elastic
    ],
    "MAPE": [
        mape_test_lr,
        mape_test_ridge,
        mape_test_lasso,
        mape_test_elastic
    ],
    "R^2": [
        r2_lr,
        r2_ridge,
        r2_lasso,
        r2_elastic
    ]
}

# Convert to DataFrame
evaluation_df = pd.DataFrame(metrics)
evaluation_df

In [ ]:
#Koeficienty
ridge_coeffs = lasso_model.coef_
feature_names = X_train.columns
ridge_coeffs_df = pd.DataFrame(ridge_coeffs, index=feature_names, columns=['Koeficient'])
print("Intercept:", ridge_model.intercept_)
ridge_coeffs_df


In [ ]:
#Faktor inflácie rozptylu

vif_data = pd.DataFrame()
vif_data["Premenná"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]


vif_data.sort_values(by="VIF", ascending=False)


In [ ]:
#Krížová validácia
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.5)
}


k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
    cv_results[name] = {
        "Priemerné R²": np.mean(scores),
        "Odchýlka R²": np.std(scores)
    }


for model, result in cv_results.items():
    print(f"{model}: Priemerné R² = {result['Priemerné R²']:.4f}, Odchýlka = {result['Odchýlka R²']:.4f}")


## Nezamestnanosť

In [ ]:
#Rozdelenie a štandardizácia dát
#X = df_clean[df_clean.columns.difference(["nezamestnanost", "stvrtrok", "dovoz", "vyvoz", "pracovna_sila", "urok", "hicp", "vladne_vydavky", "tvorba_kapitalu" ])]
X = df_clean[df_clean.columns.difference(["nezamestnanost", "stvrtrok", "dovoz", "vyvoz", "pracovna_sila", "urok", "hicp", "vladne_vydavky", "tvorba_kapitalu", "m2", "ppi", "obyvatelia"])]
y = df_clean.nezamestnanost
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

In [ ]:

# Generate synthetic dataset


# Define models
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.9)
}

# DataFrame to store results
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    # Shapiro-Wilk test for normality
    shapiro_stat, shapiro_p = shapiro(residuals)
    
    # Breusch-Pagan test for heteroscedasticity
    X_with_const = sm.add_constant(X_train)  # Add constant term
    _, bp_pvalue, _, _ = het_breuschpagan(residuals, X_with_const)
    
    # Durbin-Watson test for autocorrelation
    dw_stat = durbin_watson(residuals)
    
    # Store results
    results.append({
        "Model": name,
        "SW p-hodnota": shapiro_p,
        "BP p-hodnota": bp_pvalue,
        "DW štatistika": dw_stat
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

results_df

In [ ]:
#Lineárna regresia
mlr = LinearRegression()
mlr.fit(X_train, y_train)
y_pred_test = mlr.predict(X_test)
y_pred_train = mlr.predict(X_train)


np.set_printoptions(precision=2)
print(np.concatenate((np.array(y_pred_test).reshape(len(y_pred_test), 1), np.array(y_test).reshape(len(y_test), 1)), 1))

r2_test = mlr.score(X_test, y_test)
r2_train = mlr.score(X_train, y_train)
print(f"R² test: {r2_test}")
print(f"R² train: {r2_train}")

residuals = y_train - y_pred_train





stat, p = shapiro(residuals)
print(f"Shapiro-Wilk Test: Stat={stat:.4f}, p={p:.4f}")
if p > 0.05:
    print("Reziduá sú približne normálne.")
else:
    print("Reziduá sa signifikantne odchyľujú od normnality.")


X_test_with_const = sm.add_constant(X_train)
bp_test_stat, bp_test_pval, _, _ = het_breuschpagan(residuals, X_test_with_const)


print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {bp_test_stat:.4f}")
print(f"p-value: {bp_test_pval:.4f}")


if bp_test_pval < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(residuals, bins=20, kde=True, ax=axs[0])
axs[0].axvline(0, color='red', linestyle='dashed') 
axs[0].set_title("Histogram of Reziduí")
axs[0].set_xlabel("Reziduá")
axs[0].set_ylabel("Počet")


axs[1].scatter(y_pred_train, residuals, alpha=0.7, edgecolor='k')
axs[1].axhline(y=0, color='red', linestyle='dashed')
axs[1].set_title("Reziduá vs. Predikované hodnoty")
axs[1].set_xlabel("Predikované hodnoty")
axs[1].set_ylabel("Reziduá")

plt.tight_layout()
plt.show()

In [ ]:
#OLS

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Step 1: Add a constant to X_train for the intercept term
X_train_const = sm.add_constant(X_train)

# Step 2: Fit the OLS regression model
ols_model = sm.OLS(y_train, X_train_const).fit()

# Step 3: Print the OLS summary
print(ols_model.summary())



In [ ]:
#Ridge
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)


y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)


ridge_r2_train = ridge_model.score(X_train, y_train)
ridge_r2_test = ridge_model.score(X_test, y_test)

print(f"Ridge Regresia R² (Train): {ridge_r2_train:.4f}")
print(f"Ridge Regresia R² (Test): {ridge_r2_test:.4f}")


ridge_residuals = y_train - y_train_pred





ridge_bp_test = het_breuschpagan(ridge_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {ridge_bp_test[0]}, p-value: {ridge_bp_test[1]}")
if ridge_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(ridge_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(ridge_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(ridge_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Lasso
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)


y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)


lasso_r2_train = lasso_model.score(X_train, y_train)
lasso_r2_test = lasso_model.score(X_test, y_test)

print(f"Lasso Regresia R² (Train): {lasso_r2_train:.4f}")
print(f"Lasso Regresia R² (Test): {lasso_r2_test:.4f}")


lasso_residuals = y_train - y_train_pred





lasso_bp_test = het_breuschpagan(lasso_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {lasso_bp_test[0]}, p-value: {lasso_bp_test[1]}")
if lasso_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(lasso_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")
fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(lasso_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(lasso_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()

In [ ]:
#Elastic net
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.9)
elastic_model.fit(X_train, y_train)


y_train_pred = elastic_model.predict(X_train)
y_test_pred = elastic_model.predict(X_test)


elastic_r2_train = elastic_model.score(X_train, y_train)
elastic_r2_test = elastic_model.score(X_test, y_test)

print(f"Elastic Net R² (Train): {elastic_r2_train:.4f}")
print(f"Elastic Net R² (Test): {elastic_r2_test:.4f}")


elastic_residuals = y_train - y_train_pred





elastic_bp_test = het_breuschpagan(elastic_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {elastic_bp_test[0]}, p-value: {elastic_bp_test[1]}")
if elastic_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(elastic_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(elastic_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(elastic_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Residuá
models = {
    "Lineárna Regresia": mlr,
    "Ridge": ridge_model,
    "Lasso": lasso_model,
    "ElasticNet": elastic_model
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, (name, model) in zip(axes, models.items()):
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    ax.scatter(y_pred, residuals, alpha=0.7, edgecolor='k')
    ax.axhline(y=0, color='red', linestyle='dashed') 
    ax.set_title(name)
    ax.set_xlabel("Predikované hodnoty")
    ax.set_ylabel("Reziduá")


plt.tight_layout()
plt.show()


In [ ]:
# Q-Q graf
residuals_lr = y_train - mlr.predict(X_train)
residuals_ridge = y_train - ridge_model.predict(X_train)
residuals_lasso = y_train - lasso_model.predict(X_train)
residuals_elastic = y_train - elastic_model.predict(X_train)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle("Q-Q Grafy", fontsize=16)

stats.probplot(residuals_lr, dist="norm", plot=axes[0, 0])
axes[0, 0].set_title("Lineárna Regresia")


stats.probplot(residuals_ridge, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title("Ridge")


stats.probplot(residuals_lasso, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title("Lasso")


stats.probplot(residuals_elastic, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title("ElasticNet")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
#Hodnotiace metriky
y_pred_train_lr = mlr.predict(X_train)  
y_pred_test_lr = mlr.predict(X_test)    

y_pred_train_ridge = ridge_model.predict(X_train)  
y_pred_test_ridge = ridge_model.predict(X_test)    

y_pred_train_lasso = lasso_model.predict(X_train)  
y_pred_test_lasso = lasso_model.predict(X_test)    

y_pred_train_elastic = elastic_model.predict(X_train)  
y_pred_test_elastic = elastic_model.predict(X_test)    


mse_test_lr = mean_squared_error(y_test, y_pred_test_lr)
mae_test_lr = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_lr = np.sqrt(mse_test_lr)
mape_test_lr = mean_absolute_percentage_error(y_test, y_pred_test_lr)

mse_test_ridge = mean_squared_error(y_test, y_pred_test_ridge)
mae_test_ridge = mean_absolute_error(y_test, y_pred_test_ridge)
rmse_test_ridge = np.sqrt(mse_test_ridge)
mape_test_ridge = mean_absolute_percentage_error(y_test, y_pred_test_ridge)

mse_test_lasso = mean_squared_error(y_test, y_pred_test_lasso)
mae_test_lasso = mean_absolute_error(y_test, y_pred_test_lasso)
rmse_test_lasso = np.sqrt(mse_test_lasso)
mape_test_lasso = mean_absolute_percentage_error(y_test, y_pred_test_lasso)

mse_test_elastic = mean_squared_error(y_test, y_pred_test_lr)
mae_test_elastic = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_elastic = np.sqrt(mse_test_elastic)
mape_test_elastic = mean_absolute_percentage_error(y_test, y_pred_test_elastic)



r2_lr = r2_test
r2_ridge = ridge_r2_test
r2_lasso = lasso_r2_test
r2_elastic = elastic_r2_test

metrics = {
    "Model": [
        "Lineárna Regresia",
        "Ridge",
        "Lasso",
        "ElasticNet"
    ],
    "MSE": [
        mse_test_lr,
        mse_test_ridge,
        mse_test_lasso,
        mse_test_elastic
    ],
    "MAE": [
        mae_test_lr,
        mae_test_ridge,
        mae_test_lasso,
        mae_test_elastic
    ],
    "RMSE": [
        rmse_test_lr,
        rmse_test_ridge,
        rmse_test_lasso,
        rmse_test_elastic
    ],
    "MAPE": [
        mape_test_lr,
        mape_test_ridge,
        mape_test_lasso,
        mape_test_elastic
    ],
    "R^2": [
        r2_lr,
        r2_ridge,
        r2_lasso,
        r2_elastic
    ]
}

# Convert to DataFrame
evaluation_df = pd.DataFrame(metrics)
evaluation_df

In [ ]:
#Faktor inflácie rozptylu

vif_data = pd.DataFrame()
vif_data["Premenná"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]

vif_data.sort_values(by="VIF", ascending=False)

In [ ]:
#Koeficienty
ridge_coeffs = mlr.coef_
feature_names = X_train.columns
ridge_coeffs_df = pd.DataFrame(ridge_coeffs, index=feature_names, columns=['Koeficient'])
print("Intercept:", ridge_model.intercept_)
ridge_coeffs_df

In [ ]:
#Krížová validácia
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.5)
}


k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
    cv_results[name] = {
        "Priemerné R²": np.mean(scores),
        "Odchýlka R²": np.std(scores)
    }


for model, result in cv_results.items():
    print(f"{model}: Priemerné R² = {result['Priemerné R²']:.4f}, Odchýlka = {result['Odchýlka R²']:.4f}")


## Inflácia

In [ ]:
#Rozdelenie a štandardizácia dát
X = df_clean[df_clean.columns.difference(["stvrtrok", "cpi" , "dovoz", "vyvoz", "nehnutelnosti", "olej", "tvorba_kapitalu", "pracovna_sila", "kurz_usd_eur", "obyvatelia", "vladne_vydavky", "m2", "bilancia", "hicp"])]
y = df_clean.cpi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

In [ ]:

# Generate synthetic dataset


# Define models
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.9)
}

# DataFrame to store results
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    # Shapiro-Wilk test for normality
    shapiro_stat, shapiro_p = shapiro(residuals)
    
    # Breusch-Pagan test for heteroscedasticity
    X_with_const = sm.add_constant(X_train)  # Add constant term
    _, bp_pvalue, _, _ = het_breuschpagan(residuals, X_with_const)
    
    # Durbin-Watson test for autocorrelation
    dw_stat = durbin_watson(residuals)
    
    # Store results
    results.append({
        "Model": name,
        "SW p-hodnota": shapiro_p,
        "BP p-hodnota": bp_pvalue,
        "DW štatistika": dw_stat
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

results_df

In [ ]:
#Lineárna regresia
mlr = LinearRegression()
mlr.fit(X_train, y_train)
y_pred_test = mlr.predict(X_test)
y_pred_train = mlr.predict(X_train)

np.set_printoptions(precision=2)
print(np.concatenate((np.array(y_pred_test).reshape(len(y_pred_test), 1), np.array(y_test).reshape(len(y_test), 1)), 1))

r2_test = mlr.score(X_test, y_test)
print(f"R²: {r2_test}")


residuals = y_train - y_pred_train


fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(residuals, bins=20, kde=True, ax=axs[0])
axs[0].axvline(0, color='red', linestyle='dashed') 
axs[0].set_title("Histogram of Reziduí")
axs[0].set_xlabel("Reziduá")
axs[0].set_ylabel("Počet")


axs[1].scatter(y_pred_train, residuals, alpha=0.7, edgecolor='k')
axs[1].axhline(y=0, color='red', linestyle='dashed')
axs[1].set_title("Reziduá vs. Predikované hodnoty")
axs[1].set_xlabel("Predikované hodnoty")
axs[1].set_ylabel("Reziduá")

plt.tight_layout()
plt.show()


stat, p = shapiro(residuals)
print(f"Shapiro-Wilk Test: Stat={stat:.4f}, p={p:.4f}")
if p > 0.05:
    print("Reziduá sú približne normálne.")
else:
    print("Reziduá sa signifikantne odchyľujú od normnality.")


X_test_with_const = sm.add_constant(X_train)
bp_test_stat, bp_test_pval, _, _ = het_breuschpagan(residuals, X_test_with_const)


print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {bp_test_stat:.4f}")
print(f"p-value: {bp_test_pval:.4f}")


if bp_test_pval < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


In [ ]:
#OLS

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)


X_train_const = sm.add_constant(X_train)


ols_model = sm.OLS(y_train, X_train_const).fit()


print(ols_model.summary())



In [ ]:
#Ridge
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)


y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)


ridge_r2_train = ridge_model.score(X_train, y_train)
ridge_r2_test = ridge_model.score(X_test, y_test)

print(f"Ridge Regresia R² (Train): {ridge_r2_train:.4f}")
print(f"Ridge Regresia R² (Test): {ridge_r2_test:.4f}")


ridge_residuals = y_train - y_train_pred





ridge_bp_test = het_breuschpagan(ridge_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {ridge_bp_test[0]}, p-value: {ridge_bp_test[1]}")
if ridge_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(ridge_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(ridge_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(ridge_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Lasso
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)


y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)


lasso_r2_train = lasso_model.score(X_train, y_train)
lasso_r2_test = lasso_model.score(X_test, y_test)

print(f"Lasso Regresia R² (Train): {lasso_r2_train:.4f}")
print(f"Lasso Regresia R² (Test): {lasso_r2_test:.4f}")


lasso_residuals = y_train - y_train_pred





lasso_bp_test = het_breuschpagan(lasso_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {lasso_bp_test[0]}, p-value: {lasso_bp_test[1]}")
if lasso_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(lasso_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(lasso_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(lasso_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Elastic net
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.9)
elastic_model.fit(X_train, y_train)


y_train_pred = elastic_model.predict(X_train)
y_test_pred = elastic_model.predict(X_test)


elastic_r2_train = elastic_model.score(X_train, y_train)
elastic_r2_test = elastic_model.score(X_test, y_test)

print(f"Elastic Net R² (Train): {elastic_r2_train:.4f}")
print(f"Elastic Net R² (Test): {elastic_r2_test:.4f}")


elastic_residuals = y_train - y_train_pred





elastic_bp_test = het_breuschpagan(elastic_residuals, sm.add_constant(X_train))
print("\nBreusch-Pagan Test:")
print(f"Test Statistic: {elastic_bp_test[0]}, p-value: {elastic_bp_test[1]}")
if elastic_bp_test[1] < 0.05:
    print("Reziduá vykazujú signifkantnú heteroskedasticitu.")
else:
    print("Reziduá nevykazujú signifkantnú heteroskedasticitu.")


shapiro_test_stat, shapiro_p_value = shapiro(elastic_residuals)
print("\nShapiro-Wilk Test:")
print(f"Test Statistic: {shapiro_test_stat:.4f}, p-value: {shapiro_p_value:.4f}")
if shapiro_p_value < 0.05:
    print("Reziduá sa signifikantne odchyľujú od normnality.")
else:
    print("Reziduá sú približne normálne.")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))


sns.histplot(elastic_residuals, kde=True, bins=20, ax=axs[0])
axs[0].set_title("Histogram Reziduí")
axs[0].axvline(0, color='red', linestyle='--')


sm.qqplot(elastic_residuals, line='s', ax=axs[1])
axs[1].set_title("Q-Q Graf")

plt.show()


In [ ]:
#Residuá
models = {
    "Lineárna Regresia": mlr,
    "Ridge": ridge_model,
    "Lasso": lasso_model,
    "ElasticNet": elastic_model
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, (name, model) in zip(axes, models.items()):
    y_pred = model.predict(X_train)
    residuals = y_train - y_pred
    
    ax.scatter(y_pred, residuals, alpha=0.7, edgecolor='k')
    ax.axhline(y=0, color='red', linestyle='dashed') 
    ax.set_title(name)
    ax.set_xlabel("Predikované hodnoty")
    ax.set_ylabel("Reziduá")


plt.tight_layout()
plt.show()


In [ ]:
# Q-Q graf
residuals_lr = y_train - mlr.predict(X_train)
residuals_ridge = y_train - ridge_model.predict(X_train)
residuals_lasso = y_train - lasso_model.predict(X_train)
residuals_elastic = y_train - elastic_model.predict(X_train)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle("Q-Q Grafy", fontsize=16)

stats.probplot(residuals_lr, dist="norm", plot=axes[0, 0])
axes[0, 0].set_title("Lineárna Regresia")


stats.probplot(residuals_ridge, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title("Ridge")


stats.probplot(residuals_lasso, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title("Lasso")


stats.probplot(residuals_elastic, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title("ElasticNet")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
#Hodnotiace metriky
y_pred_train_lr = mlr.predict(X_train)  
y_pred_test_lr = mlr.predict(X_test)    

y_pred_train_ridge = ridge_model.predict(X_train)  
y_pred_test_ridge = ridge_model.predict(X_test)    

y_pred_train_lasso = lasso_model.predict(X_train)  
y_pred_test_lasso = lasso_model.predict(X_test)    

y_pred_train_elastic = elastic_model.predict(X_train)  
y_pred_test_elastic = elastic_model.predict(X_test)    


mse_test_lr = mean_squared_error(y_test, y_pred_test_lr)
mae_test_lr = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_lr = np.sqrt(mse_test_lr)
mape_test_lr = mean_absolute_percentage_error(y_test, y_pred_test_lr)

mse_test_ridge = mean_squared_error(y_test, y_pred_test_ridge)
mae_test_ridge = mean_absolute_error(y_test, y_pred_test_ridge)
rmse_test_ridge = np.sqrt(mse_test_ridge)
mape_test_ridge = mean_absolute_percentage_error(y_test, y_pred_test_ridge)

mse_test_lasso = mean_squared_error(y_test, y_pred_test_lasso)
mae_test_lasso = mean_absolute_error(y_test, y_pred_test_lasso)
rmse_test_lasso = np.sqrt(mse_test_lasso)
mape_test_lasso = mean_absolute_percentage_error(y_test, y_pred_test_lasso)

mse_test_elastic = mean_squared_error(y_test, y_pred_test_lr)
mae_test_elastic = mean_absolute_error(y_test, y_pred_test_lr)
rmse_test_elastic = np.sqrt(mse_test_elastic)
mape_test_elastic = mean_absolute_percentage_error(y_test, y_pred_test_elastic)



r2_lr = r2_test
r2_ridge = ridge_r2_test
r2_lasso = lasso_r2_test
r2_elastic = elastic_r2_test

metrics = {
    "Model": [
        "Lineárna Regresia",
        "Ridge",
        "Lasso",
        "ElasticNet"
    ],
    "MSE": [
        mse_test_lr,
        mse_test_ridge,
        mse_test_lasso,
        mse_test_elastic
    ],
    "MAE": [
        mae_test_lr,
        mae_test_ridge,
        mae_test_lasso,
        mae_test_elastic
    ],
    "RMSE": [
        rmse_test_lr,
        rmse_test_ridge,
        rmse_test_lasso,
        rmse_test_elastic
    ],
    "MAPE": [
        mape_test_lr,
        mape_test_ridge,
        mape_test_lasso,
        mape_test_elastic
    ],
    "R^2": [
        r2_lr,
        r2_ridge,
        r2_lasso,
        r2_elastic
    ]
}

# Convert to DataFrame
evaluation_df = pd.DataFrame(metrics)
evaluation_df

In [ ]:
#Faktor inflácie rozptylu
vif_data = pd.DataFrame()
vif_data["Premenná"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]

vif_data.sort_values(by="VIF", ascending=False)

In [ ]:
#Koeficienty
ridge_coeffs = mlr.coef_
feature_names = X_train.columns
ridge_coeffs_df = pd.DataFrame(ridge_coeffs, index=feature_names, columns=['Koeficient'])
print("Intercept:", ridge_model.intercept_)
ridge_coeffs_df

In [ ]:
X_train.columns

In [ ]:
#Krížová validácia
models = {
    "Lineárna Regresia": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.5)
}


k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
    cv_results[name] = {
        "Priemerné R²": np.mean(scores),
        "Odchýlka R²": np.std(scores)
    }


for model, result in cv_results.items():
    print(f"{model}: Priemerné R² = {result['Priemerné R²']:.4f}, Odchýlka = {result['Odchýlka R²']:.4f}")
